# 싱글톤이란?

프로그램 실행 중에 딱 하나만 존재하는 인스턴스를 만들어야 할 때 사용하는 디자인 패턴입니다.

특정 클래스의 인스턴스를 1개만 생성되도록 보장하여 하나의 인스턴스를 재활용합니다!

## 왜 필요했을까?

- 데이터 베이스에 접속하는 세션 관리
- 메모리 관리
    - 다중으로 반복되는 패턴에서 같은 객체를 여러 개 생성해 사용하는 문제를 해결합니다
- 설정 관리자
    - 프로그램 전체에서 동일한 설정을 공유할 때
- 로깅 서비스
    - 여러 곳의 로그를 한 파일에 기록할 때
- 하드웨어 자원 접급
    - 공유 자원(프린트 등)에 대한 접근을 관리 할 때

## 파이썬 코드의 구현

<aside>
💡

파이썬에서는 __new__ 매직 메서드를 오버라이딩 하여 클래스 인스턴스 생성에 관여 할 수 있습니다.

</aside>

In [ ]:
class MySingletone:
   _instance = None
   def __new__(cls):
       if cls._instance is None:
           cls._instance = super().__new__(cls)
       return cls._instance

위 클래스를 초기화 할 때 인스턴스는 바로 생성되지 않고 위 __new__  메서드를 지나칩니다.

이곳에서 MySingletone 클래스가 가진 _instance 에 이전에 생성된 인스턴스가 있는지 확인하고, 만약 없다면(is None) 클래스의 부모 객체(Object)를 이용해 인스턴스를 만듭니다.

이후 여러 번 초기화 하더라도, 클래스가 가진 _instance를 반환하므로 새로운 인스턴스는 생성되지 않고 변수는 이미 생성된 인스턴스를 참조하는 형태가 됩니다.

In [ ]:
mysingle1 = MySingletone()
mysingle2 = MySingletone()

print(mysingle1 == mysingle2)

이는 이렇게 간단하게 확인해볼 수 있습니다.

위 스크립트 실행 시 True 가 반환됩니다.

이는 보통 해쉬 값(__hash__)으로 초기화되는 인스턴스 이름이 같다는 것을 의미하며, 결국 같은 메모리 주소를 차지하는 똑같은 인스턴스 임을 의미합니다.

# 싱글톤의 단점

싱글톤은 다음과 같은 이유로 잘 사용되지 않습니다.

1. 전역 상태
전역 변수로 선언되어 코드 흐름 추적과 디버깅을 어렵게 만든다
2. 결합도 증가
많은 코드가 싱글톤 클래스에 직접 의존하게 되어 코드 변경 및 교체가 어렵다
3. 테스트가 어렵다
    1. 상태공유
    한 테스트가 바꾼 인스턴스의 상태가 다른 테스트에 영향을 준다
    2. Mocking 어려움
    테스트를 위해 가짜 객체로 대체할 수 없으며, 이는 항상 진짜 서비스에 접속하게 된다는 의미입니다
4. 멀티스레딩의 문제
여러 스레드가 동시에 접근 할 때 인스턴스가 여러 번 생성 될 수 있으므로 스레드 Lock 등의 별도의 처리가 필요하다

# 그러니까 싱글톤 보다는~

싱글톤은 하나만 관리한다는 측면에서 좋아 보이지만 위의 단점이 너무나 크므로! 싱글톤을 써야 한다는 생각이 들 때는 이를 먼저 고려해보자

- 의존성 주입 : 필요한 객체를 외부에서 생성해 넘겨주기(Strategy Pattern_)
- 객체를 인자로 전달: 전역 접근 대신, 필요한 곳에 객체를 직접 전달하는 것이 더 명확할 수 있다

# 파이썬에서는 이렇게 쓰세요

1. 모듈 사용하기   
파이썬에서는 모듈을 처음 import 할 때 단 한 번만 로드하고, 이후에는 메모리에 올라간 객체를 재사용하므로 자연스럽게 싱글톤 역할을 합니다

2. 데코레이터 사용   
클래스에 데코레이터(@)를 붙여 싱글톤 동작을 부여합니다.

In [ ]:
def singleton(cls):
    instances = {}
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance
    
@singleton
class Singleton:
    def __init__(self, value):
        self.value = value

3. 메타클래스 사용   
클래스를 만드는 클래스로, 클래스의 생성 과정을 제어해 싱글톤을 구현한다

In [ ]:
class SingletonMeta(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

# 싱글톤으로 만들고 싶은 클래스에 메타클래스 지정
class MySingleton(metaclass=SingletonMeta):
    pass